In [ ]:
from datetime import datetime
import os
import re

# Get the current directory
# current_directory = '/home/andrew/Downloads/Predictions2'
current_directory = '/home/andrew/Videos/VIRAT_gray_0.5_2/Predictions3'

# Regular expression to extract the [name] portion
pattern = r'(\d+)_(.*?)_(.*?)_(.*?)_(\d{2}_\d{2}_\d{4}_\d{2}_\d{2}_\d{2})\.log'

# Create a dictionary to store filenames by [name]
name_to_files = {}

# List all files with a .log extension in the current directory
log_files = [file for file in os.listdir(current_directory) if file.endswith(".log")]


# Function to parse the timestamp string into a datetime object
def parse_timestamp(timestamp_str):
    return datetime.strptime(timestamp_str, '%d_%m_%Y_%H_%M_%S')


# Iterate through log files and group them by [name]
for log_file in log_files:
    match = re.search(pattern, log_file)
    if match:
        name = match.group(4)
        timestamp_str = match.group(5)
        timestamp = parse_timestamp(timestamp_str)

        if name not in name_to_files:
            name_to_files[name] = []
        name_to_files[name].append((timestamp, log_file))

# Print the grouped filenames
for name, files in name_to_files.items():
    print(f"[name]: {name}")
    sorted_files = sorted(files, key=lambda x: x[0])  # Sort by timestamp
    for _timestamp, file in sorted_files:
        print(file)

In [ ]:
from enum import Enum
import numpy as np


class InputSet:
    def __init__(self, height, width, channels):
        self.event_bitrate = None
        self.compressed_bitrate = None
        self.adder_feature_count = None
        self.adder_time = None
        self.cv_feature_count = None
        self.cv_time = None
        self.psnr = None
        self.mse = None
        self.ssim = None
        self.recon_psnr = None
        self.recon_mse = None
        self.recon_ssim = None
        self.precision = None
        self.recall = None
        self.accuracy = None
        self.num_pixels = height*width*channels
        self.num_channels = channels
        self.tps = None
        self.dt_ref = None


def process_entry(entry, input_sets):
    if 's' in entry:
        if "OpenCV" in entry['s']:
            input_sets[-1].cv_features[entry['y'], entry['x']] = True
        elif "ADDER" in entry['s']:
            input_sets[-1].adder_features[entry['y'], entry['x']] = True
        else:
            raise Exception("Malformed entry")
    elif 'psnr' in entry:
        input_sets[-1].psnr = entry['psnr']
        input_sets[-1].mse = entry['mse']
        input_sets[-1].ssim = entry['ssim']
        
def process_recon_entry(entry, input_sets, idx):
    if 'psnr' in entry:
        input_sets[idx].recon_psnr = entry['psnr']
        input_sets[idx].recon_mse = entry['mse']
        input_sets[idx].recon_ssim = entry['ssim']


import pickle

Stage = Enum('Stage', ['RAW', 'RECON', 'COMPRESS_SIZE'])

def read_logfile(path):
    objects = []
    with (
            open(
                path,
                "rb")) as openfile:
        # Read the dimensions
        dims = openfile.readline().decode('UTF-8')
        # print(dims)
        width = int(dims.split('x')[0])
        height = int(dims.split('x')[1])
        channels = int(dims.split('x')[2])

        dbg_lines = 0
        input_interval_idx = 0
        input_sets = [InputSet(height, width, channels)]
        recon = False
        input_set_index = 0
        stage = Stage.RAW
        adu_interval = None

        while True:
            try:
                entry = pickle.load(openfile)
                if type(entry) == str and "ADDER FAST:" in entry:
                    # The start of a new interval
                    time = entry.split(': ')[1]
                    input_sets[-1].adder_time = int(time)
                    continue
                elif type(entry) == str and "bps:" in entry:
                    # The bitrate
                    # The end of the interval
                    bitrate = float(entry.split(': ')[1]) / 1024.0 / 1024.0
                    input_sets[-1].event_bitrate = bitrate

                    input_sets += [InputSet(height, width, channels)]
                    input_interval_idx += 1
                    continue
                elif type(entry) == str and "RECONSTRUCTION" in entry:
                    stage = Stage.RECON
                    input_set_index = 0
                elif type(entry) == str and "Compressed adu" in entry:
                    stage = Stage.COMPRESS_SIZE
                    input_set_index = 0
                elif type(entry) == str and "OpenCV FAST:" in entry:
                    time = entry.split(': ')[1]
                    input_sets[-1].cv_time = int(time)

                elif type(entry) == str and "META:" in entry:
                    print(entry)
                    continue
                elif type(entry) == str and "Feature results:" in entry:
                    precision = pickle.load(openfile)
                    input_sets[-1].precision = precision
                    recall = pickle.load(openfile)
                    input_sets[-1].recall = recall
                    accuracy = pickle.load(openfile)
                    input_sets[-1].accuracy = accuracy
                    
                elif type(entry) == int:
                    if stage == stage.RAW:
                        if input_sets[-1].adder_feature_count is None:
                            input_sets[-1].adder_feature_count = entry
                        else:
                            input_sets[-1].cv_feature_count = entry
                    elif stage == stage.COMPRESS_SIZE:
                        for i in range(input_set_index, input_set_index + adu_interval):
                            input_sets[i].compressed_bitrate = (entry * 8) / 1024 /1024 # Mb/s
                        input_set_index += adu_interval
                    else:
                        raise Exception("unexpected int")
                    
                else:
                    if stage == Stage.RAW:
                        process_entry(entry, input_sets)
                    elif stage == Stage.RECON:
                        process_recon_entry(entry, input_sets, input_set_index)
                        input_set_index += 1
                    elif stage == Stage.COMPRESS_SIZE:
                        print('here')
                        break

                    # print(entry)
                    dbg_lines += 1
                    # if dbg_lines == 100000:
                    #     break
            except pickle.UnpicklingError:
                line = openfile.readline().decode('UTF-8')
                if "Ticks per second" in line:
                    tps = int(line.split(': ')[2])
                    input_sets[-1].tps = tps
                elif "ticks per source interval" in line:
                    tpf = int(line.split(': ')[2])
                    input_sets[-1].dt_ref = tpf
                elif "t_max" in line:
                    dt_max = int(line.split(': ')[2])
                    adu_interval = int(dt_max / tpf)
                # print(line)
                # break
            except EOFError:
                break

        # Remove the last item because it might have none values
        input_sets = input_sets[:-1]
        if stage == Stage.RAW:
             raise Exception("Did not perform reconstruction!")
        return input_sets, width, height, channels, tps, tpf
    
print(sorted_files[0][1])
input_sets, width, height, channels, tps, tpf = read_logfile(current_directory + '/' + sorted_files[0][1])

print('\nAll done')
print(input_sets[0].recon_psnr)
print(input_sets[0].compressed_bitrate)
print(input_sets[30].compressed_bitrate)

print(sorted_files[1][1])
input_sets, width, height, channels, tps, tpf = read_logfile(current_directory + '/' + sorted_files[1][1])

print('\nAll done')
print(input_sets[0].recon_psnr)
print(input_sets[0].compressed_bitrate)
print(input_sets[30].compressed_bitrate)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import mpld3


def feature_count_plot_all(sets):
    sns.set(style="whitegrid")
    plt.figure(figsize=(8, 4))

    for i, input_sets in enumerate(sets):
        adder_feature_count = [obj.adder_feature_count for obj in input_sets]
        cv_feature_count = [obj.cv_feature_count for obj in input_sets]

        sns.lineplot(x=range(len(input_sets) - 1), y=adder_feature_count[:-1], marker=None, label="ADDER_" + str(i))
        sns.lineplot(x=range(len(input_sets) - 1), y=cv_feature_count[:-1], marker=None, label="OpenCV_" + str(i))

    plt.ylabel("# features")
    plt.xlabel("Input frame")
    plt.title("Feature count over time")
    plt.show()
    # interactive_plot = mpld3.display()
    # interactive_plot


def feature_speed_plot_all(sets):
    sns.set(style="whitegrid")
    plt.figure(figsize=(8, 4))

    assert sets[0][4].adder_time is None
    for i, input_sets in enumerate(sets[1::2]):
        adder_times = [obj.adder_time for obj in input_sets]
        cv_times = [obj.cv_time for obj in input_sets]

        sns.lineplot(x=range(len(input_sets)), y=adder_times, marker=None, label="ADDER_" + str(i))
        sns.lineplot(x=range(len(input_sets)), y=cv_times, marker=None, label="OpenCV_" + str(i))
    plt.ylabel("Time (ns)")
    plt.xlabel("Input frame")
    plt.yscale('log')
    plt.title("Feature detection speed")

    plt.show()
    # interactive_plot = mpld3.display()
    # interactive_plot

    cv_mean = np.mean(np.array(cv_times))
    adder_mean = np.mean(np.array(adder_times))
    percent_change = (adder_mean / cv_mean) - 1
    print('OpenCV mean:', cv_mean, '  Median:', np.median(np.array(cv_times)))
    print('ADDER mean:', adder_mean, "({:.1f}%)".format(percent_change * 100), '  Median:',
          np.median(np.array(adder_times)))


def bitrates_plot_all(sets, width, height, channels, tps, tpf):
    bitrates_plot_partial(sets[::2], width, height, channels, tps, tpf)
    bitrates_plot_partial(sets[1::2], width, height, channels, tps, tpf)
    
def bitrates_plot_partial(sets, width, height, channels, tps, tpf):
    sns.set(style="whitegrid")
    plt.figure(figsize=(8, 4))

    for i, input_sets in enumerate(sets):
        adder_bitrates = [obj.event_bitrate for obj in input_sets]

        total_adder_size = np.sum(adder_bitrates) / (tps / tpf)
        print(total_adder_size, "Mb")

        source_raw_Mbps = (width * height * channels * (tps / tpf) * 8.0) / 1024.0 / 1024.0
        total_raw_size = (width * height * channels * len(adder_bitrates) * 8.0) / 1024.0 / 1024.0
        print(total_raw_size, "Mb")
        framed_bitrates = [source_raw_Mbps for obj in input_sets]

        sns.lineplot(x=range(len(input_sets)), y=adder_bitrates, marker=None, label="ADDER_" + str(i))
        sns.lineplot(x=range(len(input_sets)), y=framed_bitrates, marker=None, label="OpenCV_" + str(i))
    plt.ylabel("Raw bitrate (Mb/s)")
    plt.xlabel("Input frame")
    # plt.yscale('log')
    plt.title("Bitrate")

    plt.show()

def quality_plot_all(sets):
    quality_plot_partial(sets[::2])  # Without features
    plt.title("Quality metrics-raw_nofeatures")
    
    quality_plot_partial(sets[1::2]) # With features
    plt.title("Quality metrics-raw_features")
    print("recon quality:")
    quality_plot_partial(sets[::2],recon=True)  # Without features
    plt.title("Quality metrics-recon_nofeatures")
    quality_plot_partial(sets[1::2],recon=True) # With features
    plt.title("Quality metrics-recon_features")
    
    plt.show()

def quality_plot_partial(sets, recon=False):
    sns.set(style="whitegrid")
    
    # PSNR
    plt.figure(figsize=(8, 4))
    for i, input_sets in enumerate(sets):
        if recon:
            adder_mse = [obj.recon_mse for obj in input_sets]
        else: 
            adder_mse = [obj.mse for obj in input_sets]
        sns.lineplot(x=range(len(input_sets)), y=adder_mse, marker=None, label="MSE_" + str(i))
    plt.ylabel("Score")
    plt.xlabel("Input frame")
    plt.title("Quality metrics")
    
    # # SSIM
    # plt.figure(figsize=(8, 4))
    # for i, input_sets in enumerate(sets):
    #     if recon:
    #         adder_ssim = [obj.recon_ssim for obj in input_sets]
    #     else: 
    #         adder_ssim = [obj.ssim for obj in input_sets]
    #     
    #     sns.lineplot(x=range(len(input_sets)), y=adder_ssim, marker=None, label="SSIM_" + str(i))
    # plt.ylabel("Score")
    # plt.xlabel("Input frame")
    # plt.title("Quality metrics")

    

In [ ]:
EXPECTED_VIDEO_NUM = 8 # We expect to run 8 permutations: quality 0,3,6,9 both with and without feature detection

for name, files in name_to_files.items():
    print(f"[name]: {name}")
    sorted_files = sorted(files, key=lambda x: x[0])  # Sort by timestamp
    current_input_sets = []
    
    video_count = 0
    for _, file in sorted_files:
        try:
            input_sets, width, height, channels, tps, tpf = read_logfile(current_directory + '/' + file)
            current_input_sets = current_input_sets + [input_sets]
            video_count += 1
        except Exception as e:
            break
    if video_count < 8:
        print("SKIPPING", name)
        continue

    feature_count_plot_all(current_input_sets)
    feature_speed_plot_all(current_input_sets)
    bitrates_plot_all(current_input_sets, width, height, channels, tps, tpf)
    quality_plot_all(current_input_sets)
    
    # plt.figure(figsize=(8, 4))
    # sns.set(style="whitegrid")
    # plot_mean_bitrate(current_input_sets)
    # plt.show()
    break

## All mean bitrates

In [ ]:
palette = "husl"
COLOR_MAP = sns.color_palette(palette, 4)


def plot_mean_bitrate(sets, feat=False, recon=False):
    if feat:
        sets = sets[1::2]
    else:
        sets = sets[::2]
    return plot_mean_bitrate_partial(sets, recon)
    
def plot_mean_bitrate_partial(sets, recon=False):
    running_bitrate = 0

    mean_bitrates = np.zeros((len(sets)))
    mean_qualities = np.zeros((len(sets)))

    for i, input_sets in enumerate(sets):
        # Get the mean adder bitrate
        if recon:
            adder_bitrates = [obj.compressed_bitrate for obj in input_sets if obj.compressed_bitrate is not None]
            adder_quality = [obj.recon_psnr for obj in input_sets if obj.recon_psnr is not None]
        else:
            adder_bitrates = [obj.event_bitrate for obj in input_sets if obj.event_bitrate is not None]
            adder_quality = [obj.psnr for obj in input_sets if obj.psnr is not None]
            
        mean_bitrate = np.sum(adder_bitrates)/ len(adder_bitrates)
        adder_quality = [min(x, 50.0) for x in adder_quality] # TODO: Mention in paper that we cap PSNR at 50 for practical reasons
        mean_quality = np.sum(adder_quality)/ len(adder_quality)
        # if mean_quality < 1.0:
        #     mean_quality = 1.0
        running_bitrate += mean_bitrate 
        
        if i > 0:
            # plot the line
            plt.plot([last_bitrate, mean_bitrate], [last_quality, mean_quality], c=COLOR_MAP[i], label='Line', alpha=0.5)
        # plot this as a single point
        plt.scatter( mean_bitrate, mean_quality, c=COLOR_MAP[i], marker='o', label='Point', alpha=0.5)   
        
        mean_bitrates[i] = mean_bitrate
        mean_qualities[i] = mean_quality
        
        last_bitrate = mean_bitrate
        last_quality = mean_quality
    
    return mean_bitrates, mean_qualities
        

    
    
    
    
    

In [ ]:
EXPECTED_VIDEO_NUM = 8 # We expect to run 8 permutations: quality 0,3,6,9 both with and without feature detection

def new_fig_setup_bitrate_quality(feat=False, recon=False):
    plt.figure(figsize=(8, 4))
    sns.set(style="darkgrid")
    sns.set_context("paper")
    #Make the xaxis log
    plt.xscale('log')
    plt.xlim(0.1, 510)
    xticks = [0.1, 1, 10, 100, 500]
    plt.xticks(xticks, xticks)
    plt.xlabel("Mean bitrate (Mb/s)")
    
    plt.ylim(25, 52.5)
    plt.ylabel("Mean PSNR (dB)")

    
    # Set the title
    if feat:
        if recon:
            plt.title("Mean bitrate vs. quality (feature detection + source-modeled compression)")
        else:
            plt.title("Mean bitrate vs. quality (feature detection)")
    else:
        if recon:
            plt.title("Mean bitrate vs. quality (source-modeled compression)")
        else:
            plt.title("Mean bitrate vs. quality")

    all_mean_bitrates = []
    all_mean_qualities = []
    h265_bitrates = []

    for name, files in name_to_files.items():
        sorted_files = sorted(files, key=lambda x: x[0])  # Sort by timestamp
        current_input_sets = []
        
        video_count = 0
        for _, file in sorted_files:
            try:
                input_sets, width, height, channels, tps, tpf = read_logfile(current_directory + '/' + file)
                current_input_sets = current_input_sets + [input_sets]
                video_count += 1
            except Exception as e:
                print(e)
                break
        if video_count < 8:
            print("SKIPPING", name)
            continue
        mean_bitrates, mean_qualities = plot_mean_bitrate(current_input_sets, feat=feat, recon=recon)
        
        # Get the size of the h265-encoded mp4 file
        h265_size = os.path.getsize(current_directory + '/../' + name[:-1] + '.mp4')
        h265_size_per_frame = h265_size / len(input_sets)
        h265_bits_per_sec = h265_size_per_frame * 8 * (tps / tpf)
        h265_bitrates += [h265_bits_per_sec / 1024 / 1024] #Mb/s
        
        all_mean_bitrates += [mean_bitrates]
        all_mean_qualities += [mean_qualities]
    return all_mean_bitrates, all_mean_qualities, h265_bitrates
        
            

mean_bitrates, mean_qualities, _ = new_fig_setup_bitrate_quality(feat=False, recon=False)
plt.show()

mean_bitrates_feat, mean_qualities_feat, _ = new_fig_setup_bitrate_quality(feat=True, recon=False)
plt.show()


# recon
mean_bitrates_recon, mean_qualities_recon, _ = new_fig_setup_bitrate_quality(feat=False, recon=True)
plt.show()

mean_bitrates_feat_recon, mean_qualities_feat_recon, h265_bitrates = new_fig_setup_bitrate_quality(feat=True, recon=True)
plt.show()


# Determine the change in quality and size between regular and reconstruction
mean_changes_bitrate = (np.array(mean_bitrates_recon)/np.array(mean_bitrates))-1.0
mean_changes_quality = np.array(mean_qualities_recon) - np.array(mean_qualities)
print("Mean bitrate change % 0:", np.mean(mean_changes_bitrate[0::4]))
print("Mean bitrate change % 1:", np.mean(mean_changes_bitrate[1::4]))
print("Mean bitrate change % 2:", np.mean(mean_changes_bitrate[2::4]))
print("Mean bitrate change % 3:", np.mean(mean_changes_bitrate[3::4]))
print("Mean quality change 0:", np.mean(mean_changes_quality[0::4]))
print("Mean quality change 1:", np.mean(mean_changes_quality[1::4]))
print("Mean quality change 2:", np.mean(mean_changes_quality[2::4]))
print("Mean quality change 3:", np.mean(mean_changes_quality[3::4]))


mean_changes_bitrate_h265_to_adder_0 = (np.array(mean_bitrates_recon)[:, 0]/np.array(h265_bitrates))-1.0
mean_changes_bitrate_h265_to_adder_1 = (np.array(mean_bitrates_recon)[:, 1]/np.array(h265_bitrates))-1.0
mean_changes_bitrate_h265_to_adder_2 = (np.array(mean_bitrates_recon)[:, 2]/np.array(h265_bitrates))-1.0
mean_changes_bitrate_h265_to_adder_3 = (np.array(mean_bitrates_recon)[:, 3]/np.array(h265_bitrates))-1.0

print("Mean bitrate change % 0:", np.mean(mean_changes_bitrate_h265_to_adder_0))
print("Mean bitrate change % 1:", np.mean(mean_changes_bitrate_h265_to_adder_1))
print("Mean bitrate change % 2:", np.mean(mean_changes_bitrate_h265_to_adder_2))
print("Mean bitrate change % 3:", np.mean(mean_changes_bitrate_h265_to_adder_3))
# 
# data = pd.DataFrame(columns=['Category', 'Value'])
# sns.boxplot(y='Category', x='Value', orient="h", data=data, hue="Category", palette=COLOR_MAP)
#     quartiles = data.groupby('Category')['Value'].quantile([0.25, 0.5, 0.75])
#     print("Quartiles for each category:")
#     print(quartiles)

In [ ]:
import pandas as pd
import math

EXPECTED_VIDEO_NUM = 8 # We expect to run 8 permutations: quality 0,3,6,9 both with and without feature detection

def get_feature_bitrate_quality_effect(sets, sets_features, data, recon=False):
    
    # categories = [3, 6, 9]
    
    for (i, input_sets), input_sets_features in zip(enumerate(sets), sets_features):
        # Get the mean adder bitrate
        if recon:
            adder_bitrates = [obj.compressed_bitrate for obj in input_sets if obj.compressed_bitrate is not None]
            adder_quality = [obj.recon_psnr for obj in input_sets if obj.recon_psnr is not None]
            adder_quality_mse = [obj.recon_mse for obj in input_sets if obj.recon_mse is not None]
            adder_bitrates_features = [obj.compressed_bitrate for obj in input_sets_features if obj.compressed_bitrate is not None]
            adder_quality_features = [obj.recon_psnr for obj in input_sets_features if obj.recon_psnr is not None]
            adder_quality_features_mse = [obj.recon_mse for obj in input_sets_features if obj.recon_mse is not None]
        else:
            adder_bitrates = [obj.event_bitrate for obj in input_sets if obj.event_bitrate is not None]
            adder_quality = [obj.psnr for obj in input_sets if obj.psnr is not None]
            adder_quality_mse = [obj.mse for obj in input_sets if obj.mse is not None]
            adder_bitrates_features = [obj.event_bitrate for obj in input_sets_features if obj.event_bitrate is not None]
            adder_quality_features = [obj.psnr for obj in input_sets_features if obj.psnr is not None]
            adder_quality_features_mse = [obj.mse for obj in input_sets_features if obj.mse is not None]
            
        mean_bitrate = np.sum(adder_bitrates)/ len(adder_bitrates)
        adder_quality = [min(x, 48.15) for x in adder_quality] # TODO: Mention in paper that we cap PSNR at 50 for practical reasons
        mean_quality = np.sum(adder_quality)/ len(adder_quality)
        mean_bitrate_features = np.sum(adder_bitrates_features)/ len(adder_bitrates_features)
        adder_quality_features = [min(x, 48.15) for x in adder_quality_features] # TODO: Mention in paper that we cap PSNR at 50 for practical reasons
        mean_quality_features = np.sum(adder_quality_features)/ len(adder_quality_features)
        
        mean_quality_mse = np.mean(np.array(adder_quality_mse))
        
        
        if i > 0:
            # plot the line
            # plt.plot([last_bitrate, mean_bitrate], [last_quality, mean_quality], c=COLOR_MAP[i], label='Line', alpha=0.5)
            
            # Get the predicted quality (wihout features) at the bitrate with features
            # Do logarithmic interpolation to predict the PSNR at the bitrate with features
            log_bitrate1 = math.log10(last_bitrate)
            log_bitrate2 = math.log10(mean_bitrate)
            slope = (last_quality - mean_quality) / (log_bitrate1 - log_bitrate2)
            
            
            # slope = (last_quality - mean_quality) / (last_bitrate - mean_bitrate)
            assert slope > 0
            x_diff = math.log10(mean_bitrate_features) - math.log10(mean_bitrate)
            predicted_quality = min((slope * x_diff) + mean_quality, 48.15) # Don't predict aything past our max practical psnr value
            
            # get the difference between our predicted and actual quality with feature detection
            # If this number is positive, then feature detection offers a benefit!
            quality_diff = mean_quality_features - predicted_quality
            # print(quality_diff)
            
            # Add a point to the boxplot
            new_point = {'Category': i, 'Value': quality_diff}
            data = pd.concat([data, pd.DataFrame(new_point, index=[0])], ignore_index=True)
            
        last_bitrate = mean_bitrate
        last_quality = mean_quality   
    return data

def new_fig_setup_feature_effect( recon=False):
    plt.figure(figsize=(8, 4))
    sns.set(style="darkgrid")
    sns.set_context("paper")
    #Make the xaxis log
    # plt.xscale('log')
    # plt.xlim(0.1, 510)
    # xticks = [0.1, 1, 10, 100, 500]
    # plt.xticks(xticks, xticks)
    plt.ylabel("Quality level")
    # 
    plt.xlim(-6, 4)
    plt.xlabel("Change in PSNR (dB)")

    if recon: 
        plt.title("Effect of feature detection on quality at same bitrate (reconstruction)")
    else:
        plt.title("Effect of feature detection on quality at same bitrate ")

    # Create an empty DataFrame to store the data
    data = pd.DataFrame(columns=['Category', 'Value'])

    for name, files in name_to_files.items():
        sorted_files = sorted(files, key=lambda x: x[0])  # Sort by timestamp
        current_input_sets = []
        
        video_count = 0
        for _, file in sorted_files:
            try:
                input_sets, width, height, channels, tps, tpf = read_logfile(current_directory + '/' + file)
                current_input_sets = current_input_sets + [input_sets]
                video_count += 1
            except Exception as e:
                print(e)
                break
        if video_count < 8:
            print("SKIPPING", name)
            continue
        data = get_feature_bitrate_quality_effect(current_input_sets[::2], current_input_sets[1::2], data, recon=recon)
        
    sns.boxplot(y='Category', x='Value', orient='h', data=data, hue="Category")

            

new_fig_setup_feature_effect(recon=False)
plt.show()

new_fig_setup_feature_effect(recon=True)
plt.show()


## Feature speed change overall (probably just make this a table!!)

In [ ]:
def get_feature_speed_change(sets, data, recon=False):
    
    # categories = ['opencv', 0, 3, 6, 9]
    cv_means = 0 #normalize this across the four video runs, since it's pretty much always constant
    for i, input_sets  in enumerate(sets):
        # Get the mean adder bitrate
        adder_times = [obj.adder_time for obj in input_sets]
        cv_times = [obj.cv_time for obj in input_sets]
        # percent_change = (adder_mean / cv_mean) - 1

        changes = (np.array(adder_times) / np.array(cv_times)) - 1.0
        changes *= 100.0
            
        cv_mean = np.mean(np.array(cv_times))
        cv_means += cv_mean
        # adder_mean = np.mean(np.array(adder_times))
        adder_mean = np.mean(changes)
        
        # change_sum = (np.sum(np.array(adder_times)) / np.sum(np.array(cv_times))) - 1.0
        # adder_mean = change_sum * 100.0
        
        # Add a point to the boxplot
        if i > 0: # Don't show 0 quality on here, because it's not very useful
            new_point = {'Category': i, 'Value': adder_mean}
            data = pd.concat([data, pd.DataFrame(new_point, index=[0])], ignore_index=True)
        
        # if i > 0 and adder_mean > cv_mean:
        #     print(adder_mean, cv_mean)
        
    # cv_means /= len(sets)
    # new_point = {'Category': 'OpenCV', 'Value': cv_means}
    # data = pd.concat([data, pd.DataFrame(new_point, index=[0])], ignore_index=True)
    return data     
    

def new_fig_setup_feature_speed_change():
    plt.figure(figsize=(8, 4))
    sns.set(style="darkgrid")
    sns.set_context("paper")
    #Make the xaxis log
    # plt.xscale('log')
    plt.xlim(-100, 500)
    
    # xticks = [-100, 0, 100, 200, 400, 600, 800, 1000, 1200, 1400]
    xticks = [-100, 0, 100, 200, 300, 400, 500]
    plt.xticks(xticks, xticks)
    plt.ylabel("Quality level")
    # 
    # plt.ylim(-1, 15)
    # y_ticks = [-100, -10, -1, 1, 10, 100, 1000]
    # plt.yticks(y_ticks, y_ticks)
    # plt.yscale('log')
    plt.xlabel("Change in execution time (%)")
    plt.title("Effect of ADDER transcode on feature detection speed")

    # Create an empty DataFrame to store the data
    data = pd.DataFrame(columns=['Category', 'Value'])

    for name, files in name_to_files.items():
        sorted_files = sorted(files, key=lambda x: x[0])  # Sort by timestamp
        current_input_sets = []
        
        video_count = 0
        for _, file in sorted_files:
            try:
                input_sets, width, height, channels, tps, tpf = read_logfile(current_directory + '/' + file)
                current_input_sets = current_input_sets + [input_sets]
                video_count += 1
            except Exception as e:
                print(e)
                break
        if video_count < 8:
            print("SKIPPING", name)
            continue
        data = get_feature_speed_change(current_input_sets[1::2], data)
        
    sns.boxplot(y='Category', x='Value', orient="h", data=data, hue="Category", palette=COLOR_MAP)
    quartiles = data.groupby('Category')['Value'].quantile([0.25, 0.5, 0.75])
    print("Quartiles for each category:")
    print(quartiles)

            

new_fig_setup_feature_speed_change()
plt.show()
    

## Feature speed change vs. ADDER raw event rate

In [ ]:
def get_feature_speed_change_rate(sets, data, recon=False):
    
    # categories = ['opencv', 0, 3, 6, 9]
    cv_means = 0 #normalize this across the four video runs, since it's pretty much always constant
    for i, input_sets  in enumerate(sets):
        # Get the mean adder bitrate
        
        adder_times = [obj.adder_time for obj in input_sets]
        cv_times = [obj.cv_time for obj in input_sets]
        
        
        
        changes = (np.array(adder_times) / np.array(cv_times)) - 1.0
        changes *= 100.0

        # Get the mean raw adder bitrate
        adder_bitrates = [obj.event_bitrate for obj in input_sets]
        event_rates = np.array(adder_bitrates) * 1024.0 *1024.0 / 8.0
        event_rates /= np.array([obj.num_pixels for obj in input_sets])
        if input_sets[0].num_channels == 1:
            event_rates /= 9
        else:
            event_rates /= 11
        event_rates /= (input_sets[0].tps / input_sets[0].dt_ref)
        mean_bitrate = np.sum(adder_bitrates)/ len(adder_bitrates)
        
        change_sum = np.sum(np.array(adder_times)) - np.sum(np.array(cv_times))
        # num_raw_samples =input_sets[0].num_pixels * len(input_sets)
        # print(change_sum, str(np.sum(event_rates) > num_raw_samples))
        
        mean_change = np.mean(changes)
        mean_rates = np.mean(event_rates)
        # 
        new_point = {'Category': i, 'X': mean_rates, 'Y': mean_change}
        data = pd.concat([data, pd.DataFrame(new_point, index=[0])], ignore_index=True)
        # df = pd.DataFrame({'Category': i,'X': event_rates, 'Y': changes})
        # data = pd.concat([data, df], ignore_index=True)

    return data     
    

def new_fig_setup_feature_speed_change():
    plt.figure(figsize=(8, 4))
    sns.set(style="darkgrid")
    sns.set_context("paper")
    #Make the xaxis log
    # plt.xscale('log')
    plt.xlim(0, 0.2)
    # xticks = [0.1, 1, 10, 100, 500]
    # plt.xticks(xticks, xticks)
    plt.xlabel("Events per pixel per input frame")
    # 
    plt.ylim(-100, 200)
    # y_ticks = [-100, -10, -1, 1, 10, 100, 1000, 2000]
    # plt.yticks(y_ticks, y_ticks)
    # plt.yscale('log')
    plt.ylabel("Change in execution time (%)")
    plt.title("Effect of ADDER transcode on feature detection speed")

    # Create an empty DataFrame to store the data
    data = pd.DataFrame(columns=['Category', 'Value'])

    for name, files in name_to_files.items():
        sorted_files = sorted(files, key=lambda x: x[0])  # Sort by timestamp
        current_input_sets = []
        
        video_count = 0
        for _, file in sorted_files:
            try:
                input_sets, width, height, channels, tps, tpf = read_logfile(current_directory + '/' + file)
                current_input_sets = current_input_sets + [input_sets]
                video_count += 1
            except Exception as e:
                print(e)
                break
        if video_count < 8:
            print("SKIPPING", name)
            continue
        data = get_feature_speed_change_rate(current_input_sets[1::2], data)
        
    sns.scatterplot(data=data, x="X", y="Y", hue="Category", alpha=0.5, palette=COLOR_MAP)

new_fig_setup_feature_speed_change()
plt.legend(loc='upper right')
plt.show()
    